# Libraries Importations

In [89]:
#We import the librarires

from selenium import webdriver
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time
import csv

# We define some basic and useful functions

In [90]:
def clickElement(category, classe, information):
    """This function allows you to click on an element of the HTML page"""
    element = driver.find_element_by_xpath(
        "//" + category + "[@" + classe + "='" + information + "']"
    )
    element.click()


def fillField(category, class_name, information, text):
    """This function allows you to fill in a field of the HTML page"""
    elem = driver.find_element_by_xpath(
        "//" + category + "[@" + class_name + "='" + information + "']"
    )
    elem.click()
    elem.clear()
    elem.send_keys(text)


def openLink(link):
    """This function allows you to open a link"""
    driver.get(link)
    time.sleep(6)
    

# This function will get all the href links that have the a field data-test = "job-link"
def getAll_JobDescriptionLinks():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements_by_xpath("//a[@data-test='job-link']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs

def getAll_JobDescriptionLinks_Indeed():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements_by_xpath("//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs


def assembleLinkGlassdoor(base, job_name, from_age):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.glassdoor.com/Job/machine-learning-jobs-SRCH_KO0,16.htm?fromAge=1
    job_name = job_name.replace(" ", "-")
    lenght_job_name = len(job_name)
    link = base + "Job/" + job_name + "-jobs-SRCH_KO0," + str(lenght_job_name) + ".htm?fromAge=" + str(from_age)
    return link


def assembleLinkIndeed(base, job_name, from_age, location):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.indeed.com/jobs?q=machine+learning+engineer&l=United+States&fromage=1
    job_name = job_name.replace(" ", "+")
    location = location.replace(" ", "+")
    link = base + "jobs?q=" + job_name + "&l=" + location + "&fromage=" + str(from_age)
    return link


# Glassdoor Webscrapping

In [91]:
# We set the link of the page we want to scrape
base = "https://www.glassdoor.com/"
job_names = ["machine learning engineer", "data scientist", "AI engineer"]
from_age = 1


def getJobOffer_Glassdoor(base, job_name, from_age):
    """Get all the job offers for a specific search"""
    # We create the link
    link = assembleLinkGlassdoor(base, job_name, from_age)
    # We open the link
    openLink(link)

    # We get all the job offers for this search
    job_offers = []
    previous_page_offers = []
    current_page_offers = []
    while True:
        # We get all the job offers
        current_page_offers = getAll_JobDescriptionLinks()
        # We check if the current page offers are the same as the previous page offers
        if current_page_offers == previous_page_offers:
            break
        else:
            # We add the current page offers to the job offers
            job_offers += current_page_offers
            # We update the previous page offers
            previous_page_offers = current_page_offers
            # We click on the next button
            try:
                clickElement("span", "alt", "Close")
                time.sleep(1)
            except:
                pass
            clickElement("button", "aria-label", "Next")
            time.sleep(2)
    return job_offers


def main_Glassdoor(base, job_names, from_age):
    """Get all the job offers for a list of job names"""
    # We initialize the driver through Firefox
    global driver
    driver = webdriver.Firefox()
    # We get all the job offers for each job name
    job_offers = []
    for job_name in job_names:
        job_offers.append(getJobOffer_Glassdoor(base, job_name, from_age))
    # We close the driver
    driver.close()
    return job_offers


In [92]:
job_offers = main_Glassdoor(base, job_names, from_age)

print("Glassdoor Web Scraping is done.")

file_path = "../data/job_offers_glassdoor.csv"
field_names = ["link", "job_name_searched", "date", "platform"]

for job_offers, job_name in zip(job_offers, job_names):
    with open(file_path, "a", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        # If the file is empty, we write the header
        if csvfile.tell() == 0:
            writer.writeheader()
        for job_offer in job_offers:
            writer.writerow(
                {
                    "link": job_offer,
                    "job_name_searched": job_name,
                    "date": time.strftime("%Y-%m-%d"),
                    "platform": "glassdoor",
                }
            )

print("Saving the Glassdoor data in a csv file is done.")

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\3220866325.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links = driver.find_elements_by_xpath("//a[@data-test='job-link']")
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\3220866325.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_xpath(


Glassdoor Web Scraping is done.


Saving the Glassdoor data in a csv file is done.


# Indeed WebScrapping

In [93]:
# We set the link of the page we want to scrape
base = "https://www.indeed.com/"
job_names = ["machine learning engineer", "data scientist", "AI engineer"]
from_age = 1
location = "United States"

def getJobOffer_Indeed(base, job_name, from_age, location):
    """Get all the job offers for a specific search"""
    # We create the link
    link = assembleLinkIndeed(base, job_name, from_age, location)
    # We open the link
    openLink(link)
    time.sleep(2)

    # We get all the job offers for this search
    job_offers = []

    while True:
        # We get all the job offers of the first page
        current_page_offers = getAll_JobDescriptionLinks_Indeed()
        # We add the current page offers to the job offers
        job_offers += current_page_offers

        # We try to click on the next button
        try:
            clickElement("a", "data-testid", "pagination-page-next")
            time.sleep(2)
        except:
            break

    return job_offers


def main_Indeed(base, job_names, from_age, location):
    """Get all the job offers for a list of job names"""
    # We initialize the driver through Firefox
    global driver
    driver = uc.Chrome(use_subprocess=True)
    driver.maximize_window() 
    # We get all the job offers for each job name
    job_offers = []
    for job_name in job_names:
        job_offers.append(getJobOffer_Indeed(base, job_name, from_age, location))
    # We close the driver
    driver.close()
    return job_offers



In [94]:
job_offers = main_Indeed(base, job_names, from_age, location)

print("Indeed Web Scraping is done.")

file_path = "../data/job_offers_indeed.csv"
field_names = ["link", "job_name_searched", "date", "platform"]

for job_offers, job_name in zip(job_offers, job_names):
    with open(file_path, "a", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        # If the file is empty, we write the header
        if csvfile.tell() == 0:
            writer.writeheader()
        for job_offer in job_offers:
            writer.writerow(
                {
                    "link": job_offer,
                    "job_name_searched": job_name,
                    "date": time.strftime("%Y-%m-%d"),
                    "platform": "indeed",
                }
            )

print("Saving the Indeed data in a csv file is done.")

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\3220866325.py:37: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links = driver.find_elements_by_xpath("//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_18048\3220866325.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_xpath(


Indeed Web Scraping is done.
Saving the Indeed data in a csv file is done.
